<a href="https://colab.research.google.com/github/jay-jojo-cheng/deep-tsundoku/blob/master/notebooks/Inference_with_DONUT_for_book_spines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

First, we install 🤗 Transformers, Datasets and SentencePiece (the latter is used for tokenization).

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 4.7 MB/s 
     |████████████████████████████████| 7.0 MB 57.4 MB/s 


In [2]:
!pip install -q datasets sentencepiece

     |████████████████████████████████| 431 kB 5.0 MB/s 
     |████████████████████████████████| 1.3 MB 63.0 MB/s 
     |████████████████████████████████| 212 kB 65.7 MB/s 
     |████████████████████████████████| 115 kB 69.6 MB/s 
     |████████████████████████████████| 127 kB 47.6 MB/s 


## Load image

In [3]:
!gdown 1MYUzZM4bHtSFjgW_QWSG8MOPFEcizxo_

Downloading...
From: https://drive.google.com/uc?id=1MYUzZM4bHtSFjgW_QWSG8MOPFEcizxo_
To: /content/book-spines-200.zip
100% 15.9M/15.9M [00:00<00:00, 98.7MB/s]


In [4]:
!unzip -q book-spines-200.zip

In [5]:
import pandas as pd
test_df = pd.read_csv("/content/book-spines-200/spines-200.csv", header = None, converters={0: str})
test_df.columns = ['id', 'title', 'author']

In [6]:
# we remove the double quotes since that will mess up the JSON later
import string
import re

def normalize_whitespace(str):
    str = str.strip()
    str = re.sub(r'\s+', ' ', str)
    return str


test_df['ground_truth'] = ['''{"gt_parse": {"text_sequence" : "%s"}}''' % (normalize_whitespace(title).translate(str.maketrans('', '', string.punctuation))) for title in test_df['title']]

In [27]:
# # THIS IS THE VERSION THAT USES BOTH TITLE AND AUTHOR WE DON'T USE THIS FOR NOW

# # we remove the double quotes since that will mess up the JSON later
# import string
# import re

# def normalize_whitespace(str):
#     str = str.strip()
#     str = re.sub(r'\s+', ' ', str)
#     return str


# test_df['ground_truth'] = ['''{"gt_parse": {"text_sequence" : "%s"}}''' % (normalize_whitespace(title).translate(str.maketrans('', '', string.punctuation))) for title in test_df['title']]

In [7]:
test_df

,id,title,author,ground_truth
0,001,Pride and Prejudice,Jane Austen,"{""gt_parse"": {""text_sequence"" : ""Pride and Pre..."
1,002,Nineteen Eighty-Four,George Orwell,"{""gt_parse"": {""text_sequence"" : ""Nineteen Eigh..."
2,003,Crime and Punishment,Fyodor Dostoevsky,"{""gt_parse"": {""text_sequence"" : ""Crime and Pun..."
3,004,Hamlet Macbeth Othello Romeo & Juliet,William Shakespeare,"{""gt_parse"": {""text_sequence"" : ""Hamlet Macbet..."
4,005,One Hundred Years of Solitude,Gabriel Garcia Márquez,"{""gt_parse"": {""text_sequence"" : ""One Hundred Y..."
...,...,...,...,...
195,196,Running in the Family,Michael Ondaatje,"{""gt_parse"": {""text_sequence"" : ""Running in th..."
196,197,From the Land of Green Ghosts A Burmese Odyssey,Pascal Khoo Thwe,"{""gt_parse"": {""text_sequence"" : ""From the Land..."
197,198,Chef,Jaspreet Singh,"{""gt_parse"": {""text_sequence"" : ""Chef""}}"
198,199,Where the Crawdads Sing,Delia Owens,"{""gt_parse"": {""text_sequence"" : ""Where the Cra..."


In [14]:
# new
from datasets.dataset_dict import DatasetDict
from datasets import Dataset, Image

test_dataset = Dataset.from_dict({"image": "book-spines-200/" + test_df['id'] + ".jpg",
                            #  "asin": test_df['asin'],
                             "ground_truth": test_df['ground_truth']}).cast_column("image", Image())
dataset = DatasetDict({"test": test_dataset})

In [135]:
image = dataset['test'][5]['image']
# image = image.rotate(90, expand=True) # manually rotate for now
image

## Load model and processor

In [55]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("jay-jojo-cheng/donut-cover")
model = VisionEncoderDecoderModel.from_pretrained("jay-jojo-cheng/donut-cover")

Downloading:   0%|          | 0.00/362 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/489 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/355 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809M [00:00<?, ?B/s]

## Prepare using processor

We prepare the image for the model using `DonutProcessor`.

In [136]:
pixel_values = processor(image, return_tensors="pt").pixel_values
print(pixel_values.shape)

torch.Size([1, 3, 1280, 960])


## Generate

Finally, we let the model autoregressively generate the structured data.

In [137]:
import torch

task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

outputs = model.generate(pixel_values.to(device),
                               decoder_input_ids=decoder_input_ids.to(device),
                               max_length=model.decoder.config.max_position_embeddings,
                               early_stopping=True,
                               pad_token_id=processor.tokenizer.pad_token_id,
                               eos_token_id=processor.tokenizer.eos_token_id,
                               use_cache=True,
                               num_beams=1,
                               bad_words_ids=[[processor.tokenizer.unk_token_id]],
                               return_dict_in_generate=True,
                               output_scores=True,)

In [138]:
import re

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print(sequence)

Anna Kaneina


## Convert to JSON

We can convert the generated sequence to JSON if required:

In [139]:
processor.token2json(sequence)

{'text_sequence': 'Anna Kaneina'}